# Import libs

In [1]:
import sys
import os
sys.path.append('..')
from eflow.foundation import DataPipeline,DataFrameTypes
from eflow.data_analysis import FeatureAnalysis, NullAnalysis
from eflow.model_analysis import ClassificationAnalysis
from eflow.data_pipeline_segments import FeatureTransformer, TypeFixer, DataEncoder
from eflow.utils.modeling_utils import optimize_model_grid
from eflow.utils.eflow_utils import get_type_holder_from_pipeline, remove_unconnected_pipeline_segments
from eflow.utils.math_utils import get_unbalanced_threshold
from eflow.utils.sys_utils import create_dir_structure
from eflow.utils.eflow_utils import create_color_dict_for_features
from eflow.utils.pandas_utils import data_types_table, value_counts_table, suggest_removal_features 

import pandas as pd
import numpy as np
import scikitplot as skplt
from nltk.corpus import words
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import copy
from IPython.display import clear_output
from IPython.core.getipython import get_ipython
import ipython_blocking
import nltk

In [2]:
# # Additional add ons
# !pip install pandasgui
# !pip install pivottablejs
# clear_output()

In [3]:
%matplotlib notebook
%matplotlib inline

## Juypter notebook generating cells

### Important Note: Replace if set to True will remove all the contents of whatever cell it is called in. But it can be undone with a simple CMD + Z. 🙂

In [4]:
# Author: http://tinyurl.com/y6mghyzl
def create_new_cell(contents,
                    replace=False):
    """
    Desc:
        Creates a new jupyter cell.
    """
    shell = get_ipython()
    shell.set_next_input(contents,
                         replace=replace)

def __format_list_to_string(list_name,
                            list_contents):
    """
    Desc:
        Converts a list to a string and adds newlines for formating.
    """
    output_str = f"{list_name} = ["
    escape_seq_count = 0
    final_index = len(list_contents) - 1
    req_spacing = len(output_str)

    for i,element in enumerate(list_contents):
        if i == final_index:
            output_str += f'\"{element}\"'
        else:
            output_str += f'\"{element}\",'
        
        if len(output_str.split("\n")[escape_seq_count]) > 78:
            output_str += "\n"
            output_str += (" " * req_spacing)
            escape_seq_count += 1
    output_str += "]"
    return output_str

def create_new_cell_with_removal_features(df,
                                          replace=True):
    """
    Desc:
        Creates a new cell block with a list of suggested features to remove.
    
    Args:
        df:
            Pandas DataFrame object
            
        replace:
            Boolean to determine replacing the current cell.
    """
    
    # Get suggestions for removal
    cell_content = __format_list_to_string("removal_features",
                                           suggest_removal_features(df))
    # Add a sort of calling card of the function that created it
    cell_content = f"# create_new_cell_with_removal_features(df,replace={replace})\n" + cell_content
    create_new_cell(cell_content,
                    replace=replace)

def create_new_cell_with_null_removal_features(df,
                                               null_threshold=.25,
                                               replace=True):
    """
    Desc:
        Creates a new cell block with a list of suggested features to remove based on nulls.
    
    Args:
        df:
            Pandas DataFrame object
            
        null_threshold:
            Any features that contain x% percent of nulls are suggested.
            
        replace:
            Boolean to determine replacing the current cell.
    """
    
    mis_val = df.isnull().sum()
    mis_val_percent = df.isnull().sum() / len(df)

    cell_content = __format_list_to_string("remove_null_features",
                                            mis_val_percent[mis_val_percent > null_threshold].index.to_list())
    # Add a calling card of the function that created it
    cell_content += f"# create_new_cell_with_null_removal_features(df,null_threshold={null_threshold},replace={replace})\n"
    create_new_cell(cell_content,
                    replace=replace)

def create_new_cell_with_feature_value_color_dict(df,
                                                  df_features,
                                                  value_limit=50,
                                                  replace=True):
    """
    Desc:
        Creates a new cell block with a dict of suggested feature value colors.
    
    Args:
        df:
            Pandas DataFrame object
        
        df_features:
            DataFrameTypes object.
        
        null_threshold:
            Any features that contain x% percent of nulls are suggested.
            
        value_limit:
            Limit the amount of feature_values until the system will ignore
            the feature all together for dict generation.
            
        replace:
            Boolean to determine replacing the current cell.
    """
    feature_value_color_dict = create_color_dict_for_features(df,
                                                              df_features,
                                                              value_limit)
    # Add a sort of calling card of the function that created it
    cell_content = ""
    cell_content += f"# create_new_cell_with_feature_value_color_dict(df,df_features,value_limit={value_limit},replace={replace})\n"
    cell_content += "feature_value_color_dict=dict()"
    feature_count = 0
    for feature_name, feature_value_color in feature_value_color_dict.items():
        if feature_value_color_dict[feature_name].keys(): 
            cell_content += f"\nfeature_value_color_dict[\"{feature_name}\"] = dict()"
        else:
            cell_content += f"\n\n# The feature '{feature_name}' has to many values! Asserting assumption that you don't want to give colors to each!"
        
        for feature_value, color in feature_value_color.items():

            color = feature_value_color_dict[feature_name][feature_value]
            
            if feature_name in df_features.get_bool_features() or feature_name in df_features.get_categorical_features():
                try:
                    feature_value = int(float(feature_value))
                except:
                    pass
            
            if isinstance(feature_value,str):
                feature_value = f"\"{feature_value}\""
            else:
                feature_value = f"{feature_value}"
            
            if color is None:
                cell_content += f"\nfeature_value_color_dict[\"{feature_name}\"][{feature_value}] = None"
            else:
                cell_content += f"\nfeature_value_color_dict[\"{feature_name}\"][{feature_value}] = \"{color}\""
        cell_content += "\n"
        
    create_new_cell(cell_content,
                    replace=replace)

def create_new_cell_with_categorical_dict(df,
                                          df_features,
                                          value_limit=50,
                                          replace=True):
    """
    Desc:
        Creates a new cell block with a dict of
    
    Args:
        df:
            Pandas DataFrame object

        df_features:
            DataFrameTypes object.

        value_limit:
            Limit the amount of feature_values until the system will ignore
            the feature all together for dict generation.
            
        replace:
            Boolean to determine replacing the current cell.
    """

    cell_content = ""
    cell_content += f"# create_new_cell_with_categorical_dict(df,df_features,value_limit={value_limit},replace={replace})\n"
    cell_content += "categorical_value_dict = dict()\n"
    
    categorical_value_dict = dict()
    for feature_name in df_features.get_categorical_features():
        
        # Find and sort feature values
        feature_values = df[feature_name].value_counts(sort=False).index.to_list()
        feature_values = [str(val) for val in feature_values]
        feature_values.sort()
        
        # Create feature cat dict
        cat_found = False
        categorical_value_dict[feature_name] = dict()
        for val in feature_values:
            try:
                categorical_value_dict[feature_name][int(val)] = ""
                cat_found = True
            except ValueError:
                pass
        
        # Delete feature name if no categories are found
        if not cat_found:
            del categorical_value_dict[feature_name]
    
    for feature_name,cat_val_dict in categorical_value_dict.items():
        
        if len(cat_val_dict.keys()) < value_limit:
            cell_content += f"categorical_value_dict[\"{feature_name}\"]=dict()\n"
            for cat,val in cat_val_dict.items():

                if isinstance(val,str):
                    cell_content += f"categorical_value_dict[\"{feature_name}\"][{cat}] = \"{val}\"\n"
                else:
                    cell_content += f"categorical_value_dict[\"{feature_name}\"][{cat}] = {val}\n"
        else:
            cell_content += f"\n\n# The feature '{feature_name}' has to many values! Asserting assumption that you don't want to give encode to each!"

        

    create_new_cell(cell_content,
                    replace=replace)
    
    

def create_new_cell_with_value_representation(df,
                                              df_features,
                                              value_limit=50,
                                              replace=True):
    """
    Desc:
        Creates a new cell block with a dict of suggested feature value colors.
    
    Args:
        df:
            Pandas DataFrame object

        df_features:
            DataFrameTypes object.

        value_limit:
            Limit the amount of feature_values until the system will ignore
            the feature all together for dict generation.
            
        replace:
            Boolean to determine replacing the current cell.
    """
    feature_value_representation = dict()
    for feature_name in df_features.get_string_features():
        feature_value_representation[feature_name] = dict()
        for val in df[feature_name].dropna().value_counts(sort=False).index.to_list():
            if isinstance(val,str):
                if len(val) == 0:
                    continue
                if len(val) <= 3 or val not in words.words():
                    feature_value_representation[feature_name][val] = ""

                if len(feature_value_representation[feature_name].keys()) >= 50:
                    break

        if not len(feature_value_representation[feature_name].keys()):
            del feature_value_representation[feature_name]
    cell_content = ""
    cell_content += f"# create_new_cell_with_value_representation(df,df_features,value_limit={value_limit},replace={replace})\n"
    
    cell_content += "feature_value_representation = dict()\n"
    for feature_name,val_repr_dict in feature_value_representation.items():
        
        if len(val_repr_dict.keys()) < value_limit:
            cell_content += f"feature_value_representation[\"{feature_name}\"] = dict()\n"
            for val,reprs in val_repr_dict.items():

                if isinstance(val,str):
                    cell_content += f"feature_value_representation[\"{feature_name}\"][\"{val}\"] = "
                else:
                    cell_content += f"feature_value_representation[\"{feature_name}\"][{val}] = "
                
                if isinstance(reprs,str):
                    cell_content += f"\"{reprs}\"\n"
                else:
                    cell_content += f"{reprs}\n"
        else:
            cell_content += f"\n\n# The feature '{feature_name}' has to many values! Asserting assumption that you don't want to give representation to to each!"
        
        cell_content += "\n"
    create_new_cell(cell_content,
                    replace=replace)

## Declare Project Variables

### Interaction required

In [5]:
dataset_path = "Datasets/MM_DEV_MASTER_FEES.csv"
dataset_name = "MM_DEV_MASTER_FEES"

# -----
peek_project_dir = f"{dataset_name}/Before Cleaning"

# -----
notebook_mode = True

# Import dataset

In [6]:
df = pd.read_csv(dataset_path)
shape_df = pd.DataFrame.from_dict({'Rows': [df.shape[0]],
                                   'Columns': [df.shape[1]]})
display(shape_df)
display(df.head(30))

,Rows,Columns
0,197,25


,fee_item_key_id,fee_item_desc,sort_value,hud_section_type_id,hud_line_number,summary_fee_item_type_id,finance_charge_yn,fee_paid_to_type_id,fee_provider_type_id,selectable_provider_yn,...,allow_freeform_provider_yn,include_on_provider_list_yn,allow_zero_value_yn,default_payee_to_provider_yn,default_tolerance_type_id,can_finance_yn,borrower_responsible_yn,seller_responsible_yn,allow_fee_total_allocation_yn,net_fund_edit_yn
0,203K_CONSULTANT_FEE,203K Consultant Fee,B,800,0839,CANNOT_SHOP,1,THIRDPARTY,FREEFORM,0,...,1.0,0.0,1,1,ZEROPCT,1,1,0,1,1
1,203K_PERMITS,203K Permit(s),C,1300,1337,CAN_SHOP,0,THIRDPARTY,FREEFORM,1,...,1.0,1.0,1,1,TENPCT,1,1,0,1,1
2,203K_UPDATE,203K Title Update,C,1100,1124,CAN_SHOP,1,THIRDPARTY,TITLE,1,...,1.0,1.0,1,1,TENPCT,1,1,0,1,1
3,ABSTRACT_FEE,Abstract Fee,C,1100,1143,CAN_SHOP,0,THIRDPARTY,TITLE,1,...,1.0,1.0,1,1,TENPCT,0,1,0,1,1
4,ADMINISTRATION_FEE,Administration Fee,A,800,0810,ORIGINATION,1,LENDER,NaN,0,...,0.0,0.0,1,1,ZEROPCT,0,1,0,1,1
5,APPLICATION_FEE,Application Fee,A,800,0818,ORIGINATION,1,LENDER,NaN,0,...,0.0,0.0,1,1,ZEROPCT,0,1,0,1,1
6,APPRAISAL FEE_2ND_APPRAISAL,Appraisal Fee - 2nd Appraisal,B,800,0820,CANNOT_SHOP,0,THIRDPARTY,APPRAISE,0,...,1.0,0.0,1,1,ZEROPCT,0,1,0,1,1
7,APPRAISAL_FEE,Appraisal Fee,B,800,0804,CANNOT_SHOP,0,THIRDPARTY,APPRAISE,0,...,1.0,0.0,1,1,ZEROPCT,0,1,0,1,1
8,APPRAISAL_RECERT_FEE,Appraisal Recert Fee,B,800,0808,CANNOT_SHOP,0,THIRDPARTY,APPRAISE,0,...,1.0,0.0,1,1,ZEROPCT,0,1,0,1,1
9,APPRAISAL_REVIEW_FEE,Appraisal Review Fee,B,800,0809,CANNOT_SHOP,0,THIRDPARTY,APPRAISE,0,...,1.0,0.0,1,1,ZEROPCT,0,1,0,1,1


In [7]:
data_types_table(df)

,Data Types
Features,
include_on_provider_list_yn,float64
allow_freeform_provider_yn,float64
fee_provider_type_change_yn,float64
cascade_fee_provider_info_yn,float64
net_fundable_yn,int64
seller_responsible_yn,int64
borrower_responsible_yn,int64
can_finance_yn,int64
default_payee_to_provider_yn,int64


## Remove/Declare any unwanted features

### Interaction required

Note: When starting a new project uncomment the function to get suggestions and then run the cell again.

In [8]:
# create_new_cell_with_removal_features(df,replace=True)
removal_features = ["fee_item_key_id",]

In [9]:
df.drop(columns=removal_features,
        inplace=True)

In [10]:
data_types_table(df)

,Data Types
Features,
include_on_provider_list_yn,float64
allow_freeform_provider_yn,float64
fee_provider_type_change_yn,float64
cascade_fee_provider_info_yn,float64
net_fundable_yn,int64
seller_responsible_yn,int64
borrower_responsible_yn,int64
can_finance_yn,int64
default_payee_to_provider_yn,int64


## Gui tools for quick analysis dataframes

Great interface; pauses the program; comment on/off at free will.
You will need to reset kernel after use more than likely.

In [11]:
# from pandasgui import show as qt_display
# qt_display(df)
# %matplotlib inline

In [12]:
# pivot_ui(df,
#          outfile_path='Piviot_Table_JS.html')

## Skim through Value Counts

In [13]:
# for feature_name in df.columns:
#     print(f'******* Feature: {feature_name} *******')
#     print(f'Type: {df[feature_name].dtype}')
#     display(value_counts_table(df,
#                                feature_name))
#     print("-------" * 4 + "\n\n")

# Mark target feature; set to None if not needed

### Interaction required

### Supervised learning problems (Can be set to None)

In [14]:
target_feature = "sort_value"

In [15]:
try:
    if target_feature:
        df[target_feature]
except KeyError:
    raise KeyError(f"The target feature \'{target_feature}\' was not found in the dataframe!"
                   + " Please select a valid feature from the dataframe")

In [16]:
if target_feature:
    print(f"Target feature '{target_feature}'")
    print("----" * 10)
    target_amount = len(df[target_feature].dropna().value_counts().index)
    value_count_df = value_counts_table(df,
                                        target_feature)
    if target_amount < 1:
        display(value_count_df)
    else:
        # Change arg 'max_binary_threshold' to see changes in threshold
        max_unbalanced_class_threshold, min_unbalanced_class_threshold = get_unbalanced_threshold(target_amount)
        
        print(f"max_unbalanced_class_threshold = {max_unbalanced_class_threshold * 100:.3f}%")
        print(f"min_unbalanced_class_threshold = {min_unbalanced_class_threshold * 100:.3f}%")
        display(value_count_df)
        index = 0
        for percentage in value_count_df["Percantage"]:
            percentage = float(percentage[:-1])/100
            if percentage >= max_unbalanced_class_threshold or percentage <= min_unbalanced_class_threshold:
                print(f"The value '{value_count_df.index.values[index]}' is causing the target feature to be unbalanced.\n" +
                      "This could cause a model to not properly generalize itself.")
                print("---" * 10 + "\n")

            index += 1

Target feature 'sort_value'
----------------------------------------
max_unbalanced_class_threshold = 18.571%
min_unbalanced_class_threshold = 13.571%


,Counts,Percantage
Unique Values,,
C,64,32.4873%
B,33,16.7513%
A,32,16.2437%
H,24,12.1827%
E,23,11.6751%
F,20,10.1523%
h,1,0.5076%


The value 'C' is causing the target feature to be unbalanced.
This could cause a model to not properly generalize itself.
------------------------------

The value 'H' is causing the target feature to be unbalanced.
This could cause a model to not properly generalize itself.
------------------------------

The value 'E' is causing the target feature to be unbalanced.
This could cause a model to not properly generalize itself.
------------------------------

The value 'F' is causing the target feature to be unbalanced.
This could cause a model to not properly generalize itself.
------------------------------

The value 'h' is causing the target feature to be unbalanced.
This could cause a model to not properly generalize itself.
------------------------------



# Load/Init DataFrameTypes object.

This object is used to store an abstracted form of what a feature 'should be' rather than what the pandas dataframe object says it is. In this case we will be specifying all features correct types.

Comment out/remove depending on how you want your design flow to be.

In [ ]:
df_features = DataFrameTypes(df,
                             ignore_nulls=True,
                             fix_numeric_features=True,
                             fix_string_features=True,
                             target_feature=target_feature,
                             notebook_mode=notebook_mode)

Type conflict found!
Feature Name: 'hud_line_number'
------------------------------
Numeric Value Info
	Numeric count: 195
	Numeric percentage: 98.985%
	Numeric values: ['1392', '1308', '1390', '1134', '1209', '1178', '0945', '0841', '0855', '0876', '1303', '1183', '0824', '1220', '1394', '1173', '1167', '0844', '1382', '1104', '1177', '0840', '0905', '1116', '1388', '1304', '0826', '0965', '0818', '0873', '1189', '1170', '1127', '0854', '1218', '1135', '0837', '1162', '1391', '1337', '0845', '1207', '0814', '1118', '1136', '1386', '1217', '0823', '0859', '1322', '0813', '1129', '0810', '0825', '0908', '0822', '1123', '1334', '1111', '0831', '0924', '1338', '0851', '1138', '0833', '0846', '1120', '1109', '1140', '1321', '0985', '0861', '0809', '0966', '1219', '1182', '1216', '0829', '0901', '1169', '0821', '0923', '1323', '0802', '0830', '1333', '1210', '1157', '1380', '1114', '1389', '1223', '1187', '1171', '1381', '1181', '0839', '1172', '0836', '0834', '0910', '1317', '0817', '1387'

## Make any changes to 'df_features' that automated type assertions messed up.

Ex: Sometimes df_features will think a feature is a category when it isn't. Move to proper types.

In [ ]:
df_features.set_feature_to_bool(feature_name=[])
df_features.set_feature_to_integer(feature_name=[])
df_features.set_feature_to_float(feature_name=[])
df_features.set_feature_to_string(feature_name=[])
df_features.set_feature_to_datetime(feature_name=[])
df_features.set_feature_to_categorical(feature_name=["hud_section_type_id"])

print("df_features types:")
df_features.display_features(display_dataframes=True,
                             notebook_mode=notebook_mode)

In [ ]:
print("df_features:")
df_features.display_features(display_dataframes=True,
                             notebook_mode=True)

In [ ]:
print("Dataframe types:")
data_types_table(df)

# Any basic manipulation of features

#### What I mean by this is say you want to represent a feature slightly different than it is currently displaying.
Note: that whatever maniuplation you do here you should bring to each notebook's section of "Any basic manipulation of features"

# Colors and palletes for features

### Interaction required

### Remove any unwanted values found or any unwanted features to be color coded.

#### Download natural language processing utils

In [ ]:
nltk.download('wordnet')
nltk.download('words')
nltk.download('punkt')

In [ ]:
# create_new_cell_with_feature_value_color_dict(df,df_features,value_limit=50,replace=True)
feature_value_color_dict=dict()

feature_value_color_dict["selectable_provider_yn"] = dict()
feature_value_color_dict["selectable_provider_yn"][0] = "#ff8585"
feature_value_color_dict["selectable_provider_yn"][1] = "#55a868"

feature_value_color_dict["allow_zero_value_yn"] = dict()
feature_value_color_dict["allow_zero_value_yn"][0] = "#ff8585"
feature_value_color_dict["allow_zero_value_yn"][1] = "#55a868"


feature_value_color_dict["amount_editable_yn"] = dict()
feature_value_color_dict["amount_editable_yn"][0] = "#ff8585"
feature_value_color_dict["amount_editable_yn"][1] = "#55a868"

feature_value_color_dict["allow_fee_total_allocation_yn"] = dict()
feature_value_color_dict["allow_fee_total_allocation_yn"][1] = "#55a868"

feature_value_color_dict["seller_responsible_yn"] = dict()
feature_value_color_dict["seller_responsible_yn"][0] = "#ff8585"
feature_value_color_dict["seller_responsible_yn"][1] = "#55a868"

feature_value_color_dict["borrower_responsible_yn"] = dict()
feature_value_color_dict["borrower_responsible_yn"][1] = "#55a868"

feature_value_color_dict["default_payee_to_provider_yn"] = dict()
feature_value_color_dict["default_payee_to_provider_yn"][1] = "#55a868"

feature_value_color_dict["description_editable_yn"] = dict()
feature_value_color_dict["description_editable_yn"][0] = "#ff8585"
feature_value_color_dict["description_editable_yn"][1] = "#55a868"

feature_value_color_dict["net_fund_edit_yn"] = dict()
feature_value_color_dict["net_fund_edit_yn"][1] = "#55a868"

feature_value_color_dict["can_finance_yn"] = dict()
feature_value_color_dict["can_finance_yn"][0] = "#ff8585"
feature_value_color_dict["can_finance_yn"][1] = "#55a868"


feature_value_color_dict["finance_charge_yn"] = dict()
feature_value_color_dict["finance_charge_yn"][0] = "#ff8585"
feature_value_color_dict["finance_charge_yn"][1] = "#55a868"

In [ ]:
from eflow.widgets import ColorLabelingWidget

cleaning_widget = ColorLabelingWidget()
cleaning_widget.run_widget(feature_value_color_dict)

In [ ]:
df_features.set_feature_colors(feature_value_color_dict)

# General Analysis of feature data

In [ ]:
feature_analysis = FeatureAnalysis(df_features,
                                   project_sub_dir=peek_project_dir)
feature_analysis.perform_analysis(df,
                                  dataset_name=dataset_name,
                                  display_visuals=False,
                                  display_print=False,
                                  suppress_runtime_errors=False)
del feature_analysis

# Null Analysis of data

In [ ]:
null_analysis = NullAnalysis(df_features,
                             project_sub_dir=peek_project_dir,
                             notebook_mode=notebook_mode)

null_analysis.perform_analysis(df,
                               dataset_name=dataset_name,
                               null_features_only=True)

del null_analysis

## Create a json file of df_features

In [ ]:
created_dir = create_dir_structure(os.getcwd(),
                                   f"/eflow Data/{dataset_name}")

In [ ]:
df_features.create_json_file_representation(created_dir,
                                            "df_features.json")

In [ ]:
df_features = DataFrameTypes()
df_features.init_on_json_file(os.getcwd() + f"/eflow Data/{dataset_name}/df_features.json")

In [ ]:
df_features.display_features()

In [ ]:
remove_unconnected_pipeline_segments()

In [ ]:
df[df["sort_value"] == "F"]

In [ ]:
df[df["sort_value"] == "F"]["hud_section_type_id"].value_counts()

In [ ]:
df["sort_value"]

In [ ]:
for val in set(df["sort_value"].values):
    for feature in df.columns:
        
        if feature == "sort_value":
            continue
        
        print(f"sort val = {val} by {feature}")
        display(value_counts_table(df[df["sort_value"] == val],
                                   feature))
        print("\n\n")
        
    print("^^^^^^" * 10)